In [1]:
import pandas as pd
import os
import psycopg2
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [2]:
# Cria conexão com o banco de dados
def conexao():
    con = psycopg2.connect(
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        database=os.getenv("POSTGRES_DB"),
        host="localhost",
    )

    return con

# Cria função para selecionar os dados
def se_dados(query):
    vcon = conexao()
    c = vcon.cursor()
    c.execute(query)
    rows = c.fetchall()
    vcon.commit()
    vcon.close()
    return rows                    

In [5]:
# Cria query para gerar o relatório
query = f" SELECT DISTINCT papel, div_liquida, cotacao, pl, vpa, ev_ebitda, marg_liquida, lucro_liquido_12m, lucro_liquido_3m \
            FROM dados \
                WHERE data_dado_inserido >= '23-04-2025' \
                    AND div_liquida < '0' \
                    AND pl > 5 \
                    AND pl < 15 \
                        ORDER BY 2 "
result = se_dados(query)

# Cria DataFrame e salva o arquivo gerado em csv
df_rl = pd.DataFrame(result)
# Renomea o nome das colunas do Dataframe
df_rl.rename(columns={0:'Papel', 
    1:'Dívida Líquida', 
    2:'Cotação',
    3:'PL',
    4:'VPA',
    5:'EV/Ebitda',
    6:'Margem Líquida',
    7:'Lucro Líquido 12m',
    8:'Lucro Líquido 3m'}, inplace=True)
df_rl.to_csv('../Analise/relatorio.csv', sep=';', encoding='ISO-8859-1')
df_rl.head(7)

,Papel,Dívida Líquida,Cotação,PL,VPA,EV/Ebitda,Margem Líquida,Lucro Líquido 12m,Lucro Líquido 3m
0,PSSA3,"-R$ 9.606.050.000,00","R$ 41,60",10.17,21.73,0.0,0.0,"R$ 2.644.850.000,00","R$ 670.765.000,00"
1,CMIN3,"-R$ 5.071.100.000,00","R$ 6,07",7.35,1.87,0.0,0.0,"R$ 4.527.720.000,00","R$ 2.016.030.000,00"
2,WHRL3,"-R$ 2.285.250.000,00","R$ 3,98",6.58,1.79,0.0,0.0,"R$ 908.716.000,00","R$ 277.786.000,00"
3,WHRL4,"-R$ 2.285.250.000,00","R$ 4,45",7.36,1.79,0.0,0.0,"R$ 908.716.000,00","R$ 277.786.000,00"
4,LREN3,"-R$ 1.825.810.000,00","R$ 12,91",11.43,10.17,0.0,0.0,"R$ 1.196.670.000,00","R$ 487.168.000,00"
5,CXSE3,"-R$ 1.209.920.000,00","R$ 16,11",12.84,4.30,0.0,0.0,"R$ 3.765.180.000,00","R$ 1.167.000.000,00"
6,GRND3,"-R$ 1.074.950.000,00","R$ 5,26",6.45,4.48,0.0,0.0,"R$ 735.236.000,00","R$ 330.383.000,00"


In [4]:
import win32com.client as win32
import pathlib

# Inserindo arquivo a ser anexado
anexo = pathlib.Path('../Analise/relatorio.csv')
anexo_ab = str(anexo.absolute())

# Criar a integração com o outlook
outlook = win32.Dispatch('outlook.application')

# Criar um email
email = outlook.CreateItem(0)

# Configurar as informações do seu e-mail
email.To = os.getenv("EMAIL")
email.Subject = " Relatório - Ações listadas na Bolsa "
email.HTMLBody = f"""

<p>Olá,</p>
<br>
<p>Segue em anexo o relatório solicitado.</p>
<br>
<p>Atenciosamente,</p>
<p>Lucas Alves.</p>
<p>Analista de Dados</p>
<br>
<img src="../Image/1-relatorio-gerencial.jpg">

"""
# Configurar anexo
email.Attachments.Add(anexo_ab)

# Envia o e-mail
email.Send()
print("Email Enviado")

Email Enviado
